In [ ]:
import tensorflow as tf

# Networks
    VGG16, ResNet

### Stack VGG16 with Keras

In [ ]:
inp = tf.keras.layers.Input([224,224,3])
x = tf.keras.layers.Conv2D(64, (3,3), padding="same")(inp)
x = tf.keras.layers.Conv2D(64, (3,3), padding="same")(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(128, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(128, (3,3), padding="same")(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(256, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(256, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(256, (3,3), padding="same")(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(512, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(512, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(512, (3,3), padding="same")(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(512, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(512, (3,3), padding="same")(x)
x = tf.keras.layers.Conv2D(512, (3,3), padding="same")(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4096)(x)
x = tf.keras.layers.Dense(4096)(x)
out = tf.keras.layers.Dense(1000)(x)

model = tf.keras.Model(inp, out)
model.compile()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0     

### VGG16 with Keras Application
    https://keras.io/ko/applications/

In [ ]:

pre_model = tf.keras.applications.VGG16()
pre_model.summary()


553467904/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

### ResNet50 with keras application

In [ ]:
pre_model = tf.keras.applications.ResNet50()
pre_model.summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

# Using Pretrained Model with our dataset
### cfar10 Dataset Example
- 10 classes
- 50000 images
- Input size of 32 x 32 x 3

In [ ]:
vgg_pre = tf.keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(32,32,3))
# include_top parameter = 마지막 Fully Connected Layer를 넣을지 말지를 결정하는 파라미터
vgg_pre.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
# Fine tuning을 위해 일부 레이어의 Freeze 해제
set_trainable = False
for layer in vgg_pre.layers[:-2]:
    layer.trainable = False

In [ ]:
x = vgg_pre.output
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512)(x)
out = tf.keras.layers.Dense(10)(x)

custom_vgg = tf.keras.Model(vgg_pre.input, out)
custom_vgg.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
custom_vgg.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
cfar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cfar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

custom_vgg.fit(x = x_train,
               y = y_train,
               epochs = 100,
               shuffle = True,
               validation_split = 0.2)

170500096/170498071 [==============================] - 2s 0us/step
Epoch 1/100
1250/1250 [==============================] - 636s 508ms/step - loss: -33.3693 - acc: 0.1017 - val_loss: -53.4024 - val_acc: 0.1009
Epoch 2/100
1250/1250 [==============================] - 624s 500ms/step - loss: -53.2812 - acc: 0.1009 - val_loss: -53.4028 - val_acc: 0.1023
Epoch 3/100
1250/1250 [==============================] - ETA: 0s - loss: -53.4505 - acc: 0.1002

KeyboardInterrupt: ignored